In [177]:
from typing import *
from pathlib import Path
import subprocess
import platform
from pygit2 import Repository
from tempfile import TemporaryDirectory
import yaml

from faststream_gen._code_generator.helper import write_file_contents, set_cwd

In [9]:
REPO_PATH = Path("./test-workflow-delete-after").resolve()

In [10]:
str(REPO_PATH)

'/work/fastkafka-gen/nbs/test-workflow-delete-after'

In [141]:
def execute_command(command: List[str], work_dir: Path) -> Optional[Tuple[str, str]]:
    with set_cwd(work_dir):
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run( # nosec: B602, B603, B607 subprocess call - check for execution of untrusted input.
            command,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        
        if p.returncode != 0:
            return (str(p.stderr.decode('utf-8')), str(p.stdout.decode('utf-8')))

In [142]:
def create_git_repo(repository_path: Path) -> Optional[str]:
    repository_path.mkdir(parents=True, exist_ok=True)     
    command = ["git", "init"]
    return execute_command(command, repository_path)
    
    

In [143]:
create_git_repo(REPO_PATH)
assert (REPO_PATH / ".git").is_dir()

In [151]:
def git_add_and_commit(repository_path: Path, commit_message: str) -> Optional[str]:
    command = ["git", "add", "."]
    errors = execute_command(command, repository_path)
    if errors:
        return errors
    
    command = ["git", "commit", "-m", f"{commit_message}"]
    stderr_stdout = execute_command(command, repository_path)
    if stderr_stdout:
        if "nothing to commit, working tree clean" in stderr_stdout[1]:
            pass
        else:
            return stderr_stdout
    

In [152]:
def create_git_branch(branch_name: str, repository_path: Path) -> Optional[str]:
    command = ["git", "checkout", "-b", f"{branch_name}"]
    stderr_stdout = execute_command(command, repository_path)
    if stderr_stdout:
        if f"fatal: A branch named '{branch_name}' already exists." in stderr_stdout[0]:
            pass
        else:
            return stderr_stdout

In [153]:
branch = "test_branch"
create_git_branch(branch, REPO_PATH)

In [157]:
initial_description = """
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.
"""
write_file_contents(str(REPO_PATH / "description.txt"), initial_description)

In [158]:
git_add_and_commit(REPO_PATH, "Add aplication description")

In [159]:
assert Repository(str(REPO_PATH)).head.shorthand == branch

In [163]:
def chat_with_user_and_update_description(initial_description: str, updated_description_file_name: str):
    """
    Chat between PO (Product Owner) and User.
    
    Once the User has answered to all the POs questions, 
    new conversation needs to be stared for the creation of brief from description and follow up questions & answers.
    Second conversation can end when the brief is created.
    """
    
    PO_USER_CONVERSATION = f"""
    User_proxy (to chat_manager):

    {initial_description}

    Is everything clear with the above description?

    Product_Owner (to chat_manager):
    Which API should we use to retrieve the cryptocurrency price data?

    User_proxy (to chat_manager):
    coinbase API

    Product_Owner (to chat_manager):
    Are there any specific cryptocurrencies we need to focus on, or should we fetch data for all?

    User_proxy (to chat_manager):
    Bitcoin and Etherum

    Product_Owner (to chat_manager):
    Everything is clear
    """
    
    # brief_proxy needs to start another conversation which then creates updated description which is saved to a file
    updated_description = """
    The task at hand is to create a FastStream application that retrieves the current price of Bitcoin and Ethereum cryptocurrencies from the Coinbase API. 
    The data should be fetched every 2 seconds with message production in JSON format, 
    having two attributes: price (current price of cryptocurrency in USD) and crypto_currency (e.g. BTC, ETH). 
    This information is to be published to the 'new_crypto_price' topic with the utf-8 encoded 'crypto_currency' attribute serving as the partition key.
    """
    write_file_contents(str(REPO_PATH / updated_description_file_name), updated_description)
    

In [164]:
updated_description_file_name = "updated_description.txt"
chat_with_user_and_update_description(initial_description, updated_description_file_name)
assert (REPO_PATH / updated_description_file_name).is_file()

In [200]:
create_venv_and_run_tests_bash_script = """
#!/bin/bash

# get the project directory from command line argument
project_dir=$1

# get the venv directory from command line argument
venv_dir=$2

# create a new venv in the specified directory
python3 -m venv $venv_dir/my_venv > /dev/null 2>&1

# activate the venv
source $venv_dir/my_venv/bin/activate

python3 -m pip install --upgrade pip

# navigate to the project directory
cd $project_dir

pip install cookiecutter

cookiecutter https://github.com/airtai/cookiecutter-faststream.git --no-input --config-file config3.yaml

# deactivate the venv
deactivate
"""

In [201]:
def setup_venv_and_use_cookiecutter(repository_path: Path, default_context: Dict[str, Dict[str, str]]) -> None:
    with TemporaryDirectory() as d:
        bash_file = repository_path / "faststream_cookiecutter.sh"
        write_file_contents(str(bash_file), create_venv_and_run_tests_bash_script)
        
        config_file = repository_path / "config3.yaml"
        with open(str(config_file), 'w') as outfile:
            yaml.dump(default_context, outfile, default_flow_style=False)
        
        write_file_contents(str(bash_file), create_venv_and_run_tests_bash_script)
        
        command = ["bash", "faststream_cookiecutter.sh", repository_path, Path(d).resolve()]
        with set_cwd(repository_path):
            # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
            p = subprocess.run( # nosec: B602, B603, B607 subprocess call - check for execution of untrusted input.
                command,
                stderr=subprocess.PIPE,
                stdout=subprocess.PIPE,
                shell=True if platform.system() == "Windows" else False,
            )

            if p.returncode != 0:
                return (str(p.stderr.decode('utf-8')), str(p.stdout.decode('utf-8')))

In [202]:
default_context = {
    "default_context": {
        "username": "rjambrecic",
        "project_name": "First app",
        "project_slug": "fs_kafka_app",
        "streaming_service": "kafka",
    }                  
}


In [204]:
setup_venv_and_use_cookiecutter(REPO_PATH, default_context)

('faststream_cookiecutter.sh: line 14: /tmp/tmpsj070uw5/my_venv/bin/activate: No such file or directory\n\x1bDEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063\x1b\x1b\n\x1b\x1bDEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063\x1b\x1b\n\x1bfaststream_cookiecutter.sh: line 26: deactivate: command not found\n',
 'Defaulting to user installation because normal site-packages is not writeable\nRequirement

In [208]:
git_add_and_commit(REPO_PATH, "Create faststream project template")

In [220]:
application_skeleton = """
import sklearn3
"""



In [209]:
#! pip install pyautogen

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [225]:
commands = {
    "checkout": ["git", "checkout", "into_branch"],
    "merge": ["git", "merge", "from_branch"],
    "delete": ["git", "branch", "--delete", "from_branch"]
}
for command in commands.values():
    print(command)

['git', 'checkout', 'into_branch']
['git', 'merge', 'from_branch']
['git', 'branch', '--delete', 'from_branch']


In [233]:
def merge_and_delete_branch(into_branch: str, from_branch: str, repository_path: Path) -> Optional[str]:
    commands = {
    "checkout": ["git", "checkout", into_branch],
    "merge": ["git", "merge", from_branch],
    "delete": ["git", "branch", "--delete", from_branch]
    }
    for command in commands.values():
        errors = execute_command(command, repository_path)
        if errors:
            print(errors)
            return errors
    

In [234]:
merge_and_delete_branch("test_branch", "delete_me_after", REPO_PATH)

('merge: delete_me_after - not something we can merge\n', '')


('merge: delete_me_after - not something we can merge\n', '')

In [257]:
from contextlib import contextmanager

@contextmanager
def git_setup(new_branch: str, repository_path: Path) -> None:
    current_branch = Repository(str(repository_path)).head.shorthand
    errors = create_git_branch(new_branch, repository_path)
    if errors:
        return errors
    
    git_parameters = {"commit_message": "My commit message", "merge_and_delete_branch": False}
    yield git_parameters
    
    git_add_and_commit(repository_path, git_parameters["commit_message"])
    if git_parameters["merge_and_delete_branch"]:
        merge_and_delete_branch(into_branch=current_branch, from_branch=new_branch, repository_path=repository_path)
        

In [236]:
with git_setup("delete_me_after2", REPO_PATH) as git_parameters:
    write_file_contents(str(REPO_PATH / "delete_me.py"), application_skeleton)
    git_parameters["commit_message"] = "Application skeleton imlemented"
    git_parameters["merge_and_delete_branch"] = True

In [267]:
def initial_commit(initial_description: str, repository_path: Path, default_branch: str="main"):
    create_git_repo(repository_path)
    create_git_branch(default_branch, repository_path)
    
    write_file_contents(str(repository_path / "description.txt"), initial_description)
    initial_commit = "Initial application description"
    git_add_and_commit(repository_path, initial_commit)

In [268]:
REPO_PATH2 = Path("/work/fastkafka-gen/nbs/test-new-repo")

initial_commit("Create a FastStream application...", REPO_PATH2)

In [269]:
with git_setup("delete_me_after", REPO_PATH2) as git_parameters:
    write_file_contents(str(REPO_PATH2 / "delete_me.py"), application_skeleton)
    git_parameters["commit_message"] = "Application skeleton imlemented"
    git_parameters["merge_and_delete_branch"] = True

In [212]:
import autogen
import os
import random

api_key = os.environ['OPENAI_API_KEY']

config_list = [
    {
        "model": "gpt-4",
        "api_key": api_key
    },
]


In [ ]:
llm_config={
    "request_timeout": 600,
    "seed": random.randint(100, 900),
    "config_list": config_list,
    "temperature": 0,
}


def git_add_and_commit_function(commit_message: str) -> Optional[str]:
    return git_add_and_commit(REPO_PATH, commit_message)

generate_skeleton_assistant = autogen.AssistantAgent(
    name="generate_skeleton_assistant",
    llm_config=llm_config,
    "functions": [
            {
                "name": "git_add_and_commit_function",
                "description": "Git add and coomit all the files changed in the repository.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "commit_message": {
                            "type": "string",
                            "description": "The git commit message",
                        },
                    },
                    "required": ["commit_message"],
                },
            },
        ],
)